In [1]:
from Load_Data_for_Modelling_Function import Data_for_Model
from Load_Data_for_Modelling_Interpolation import Interpolation

data = Data_for_Model(0,1800)

#df_Int, Interpoliertes_df = Interpolation(0,1800,2)


Anzahl der Zeilen im finalen DataFrame: 255600
None


In [2]:
from Fensterung_Scaling_CNN_ValData_Interpolation_Test import Fensterung_Scale

# Falls Validations Daten benötigt werden
X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=42, Train_Test_Split=2, window_size=10)
#X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, X_test_scaled_int, Y_test_interpolation, Blechnummern_Test_Int = Fensterung_Scale(df_Int, interpoliertesdf=Interpoliertes_df, Validation_data=1, random=8, Train_Test_Split=2, window_size=10, Interpolation=1)
# Falls keine Validationsdaten benötigt werden 
#X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled, Y_train, Y_test, scalers_features, scaler_labels = Fensterung_Scale(Validation_data=0, random=42)

142
254180
142.0
Richtige Blechnummern umgerechnet der Validierungsdaten [42, 20, 84, 76, 71, 49, 40, 36, 135, 22, 21, 37, 69, 73]
Richtige Blechnummern umgerechnet der Testdaten [78, 103, 17, 85, 39, 117, 109, 115, 83, 67, 169, 165, 101, 164]
Blechnummern für Validierungsdaten, abgezählt vom Array nicht die Originaldaten [29, 7, 71, 63, 58, 36, 27, 23, 109, 9, 8, 24, 56, 60]
Blechnummern für Testdaten, abgezählt vom Array nicht die Originaldaten[65, 78, 4, 72, 26, 92, 84, 90, 70, 54, 141, 137, 76, 136]
25060
Shape nach dem Random Sampling des Arrays von X_val: (25060, 10, 11)
Shape nach dem Random Sampling des Arrays von X_train: (204060, 10, 11)
Shape nach dem Random Sampling des Arrays von Y_train: (204060, 1, 3)
(25060, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (204060, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25060, 10, 11) 
Shape für die Labels der ge

In [4]:
print(Y_train.shape)
print(Y_val.shape)
import numpy as np
Y_train = np.squeeze(Y_train)
Y_test = np.squeeze(Y_test)
Y_val =np.squeeze(Y_val)
Y_train_scaled = np.squeeze(Y_train_scaled)
Y_val_scaled = np.squeeze(Y_val_scaled)
#Y_test_interpolation = np.squeeze(Y_test_interpolation)
# Y_train_int = np.squeeze(Y_train_int)
# Y_test_int = np.squeeze(Y_test_int)
# Y_val_int =np.squeeze(Y_val_int)
# Y_train_scaled_int = np.squeeze(Y_train_scaled_int)
# Y_val_scaled_int = np.squeeze(Y_val_scaled_int)

(204060, 1, 3)
(25060, 1, 3)


In [5]:
# Neural Transformer Network

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
from kerastuner import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import shutil
import os
import json
from tensorflow.keras.layers import Input, Embedding
import gc
from tensorflow.keras.backend import clear_session

# Define the path to save the model and hyperparameters
tuner_directory = 'my_directory'
output_path = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\14_Modelle_Hyperparameter'

# Remove the previous tuner directory if it exists
if os.path.exists(tuner_directory):
    shutil.rmtree(tuner_directory)
    
# Positional Encoding

# class PositionalEncoding(layers.Layer):
#     def __init__(self, d_model, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.pos_encoding = self.positional_encoding(max_len, d_model)

#     def get_config(self):
#         config = super().get_config().copy()
#         config.update({
#             'd_model': self.d_model,
#             'max_len': self.max_len,
#         })
#         return config
    
#     def positional_encoding(self, position, d_model):
#         angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
#                                      np.arange(d_model)[np.newaxis, :],
#                                      d_model)

#         # apply sin to even indices in the array; 2i
#         angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

#         # apply cos to odd indices in the array; 2i+1
#         angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

#         pos_encoding = angle_rads[np.newaxis, ...]

#         return tf.cast(pos_encoding, dtype=tf.float32)

#     def get_angles(self, pos, i, d_model):
#         angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
#         return pos * angle_rates

#     def call(self, inputs):
#         seq_len = tf.shape(inputs)[1]
#         return inputs + self.pos_encoding[:, :seq_len, :]
    
    
    # Transformer Encoder Layer
def transformer_encoder(inputs, head_size, num_heads, ff_dim, kernel_size1, activation_cnn ,dropout_1, dropout_2, dropout_3):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_1)(x, x)
    x = layers.Dropout(dropout_2)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size= kernel_size1, activation=activation_cnn, padding='same')(x)
    x = layers.Dropout(dropout_3)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res





def build_model(hp):
    # Define the Transformer model
    input_layer = layers.Input(shape=(10, 11))
    
    print(input_layer.shape)
    
    # Embedding Layer
    #Embedding layer using Dense
    # dense_1 = hp.Int(f'units_dense_embedding', min_value=32, max_value=512, step=32)
    # x = layers.Dense(dense_1)(input_layer)
    
    # print(x.shape)
        
    #Positional Encoding Layer hinzufügen
    # pos_encoding_layer = PositionalEncoding(dense_1, max_len=100)(x)
    
    # num_transformer_blocks = hp.Int('num_transformer_blocks', min_value=1, max_value=5, step=1)
    
    # x = pos_encoding_layer
    
    num_transformer_blocks = hp.Int('num_transformer_blocks', 0, 6)
    
    x = input_layer
    
    for i in range(num_transformer_blocks):
        
        head_size = hp.Int(f'headsize_{i}', min_value=32, max_value=512, step=32)
        num_heads = hp.Int(f'num_heads_{i}', min_value=2, max_value=6, step=1)
        ff_dim = hp.Int(f'ff_dim_{i}',min_value=32, max_value=512, step=32)
        #num_conv_layers = hp.Int(f'num_conv_layers_{i}', min_value=1, max_value=4, step=1)
        dropout_1 = hp.Float(f'dropout_rate_1_Nummer{i}', min_value=0.0, max_value=0.5, step=0.1, default=0.0)
        dropout_2 = hp.Float(f'dropout_rate_2_Nummer{i}', min_value=0.0, max_value=0.5, step=0.1, default=0.0)
        dropout_3 = hp.Float(f'dropout_rate_3_Nummer{i}', min_value=0.0, max_value=0.5, step=0.1, default=0.0)
        activation_cnn = hp.Choice(f'activation_cnn_{i}', values=['relu', 'tanh'])
        kernel_size1 = hp.Choice(f'kernel_size_CNNLayer{i}', values = [2,3,4,5])
        
        
        x = transformer_encoder(x, head_size, num_heads, ff_dim, kernel_size1, activation_cnn ,dropout_1, dropout_2, dropout_3)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)

    # Fully Connected Layers
    num_layers_fully = hp.Int('num_layers_fully', 1, 5)
    hp_layer_1 = hp.Int('layer_dense', min_value=32, max_value=512, step=32)
    hp_activation_fully1 = hp.Choice('activation_fully', values=['relu', 'tanh'])
    l2_dense = hp.Float('l2_dense', min_value=0.0, max_value=0.01, step=0.001)
    
    dense_layer = layers.Dense(hp_layer_1, activation=hp_activation_fully1, kernel_regularizer=keras.regularizers.l2(l2_dense))(x)
    for i in range(num_layers_fully - 1):
        units_dense = hp.Int(f'units_dense{i}', min_value=32, max_value=512, step=32)
        activation_layer_dense = hp.Choice(f'activation_dense{i}', values=['relu', 'tanh'])
        l2_dense_x = hp.Float(f'l2_dense{i}', min_value=0.0, max_value=0.01, step=0.001)
        
        dense_layer = layers.Dense(units_dense, activation=activation_layer_dense, kernel_regularizer=keras.regularizers.l2(l2_dense_x))(dense_layer)

    # Output Layers
    X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
    Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
    Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)

    outputs = [X_output, Y_output, Phi_output]

    model = keras.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')), 
                  loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                  metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})
    model.summary()
    
    return model 

# Instantiate the Keras Tuner
tuner = kt.BayesianOptimization(build_model,
                                objective='val_loss',
                                max_trials=30,
                                executions_per_trial=1,
                                directory=tuner_directory,
                                project_name='Transformer_Hyperparametertuning_BayesianOptimization')

# Define a custom callback to stop training if validation loss is above a threshold after 2 epochs
class EarlyStopOnHighValLoss(tf.keras.callbacks.Callback):
    def __init__(self, threshold, patience=3):
        super(EarlyStopOnHighValLoss, self).__init__()
        self.threshold = threshold
        self.patience = patience
        self.wait = 0
        
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss > self.threshold:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
        else:
            self.wait = 0
        
# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
early_stop_on_high_val_loss = EarlyStopOnHighValLoss(threshold=2, patience=3)  

# Run the hyperparameter search
tuner.search(X_train_scaled, [Y_train[:, 0], Y_train[:, 1], Y_train[:, 2]],
             epochs=20,
             validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
             callbacks=[early_stopping, early_stop_on_high_val_loss])

clear_session()
gc.collect()

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Save the best hyperparameters to a JSON file
Pfad = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\14_Modelle_Hyperparameter'
hyperparameters_pfad = os.path.join(Pfad, 'best_hyperparameters_Transformer_30Trials_Bayesian.json')
with open(hyperparameters_pfad, 'w') as json_file:
    json.dump(best_hps.values, json_file)

# Debugging: Print all hyperparameters
print("All available hyperparameters:")
print(best_hps.values)

#print(f"Best embedding_dim: {best_hps.get('embedding_dim')}")
# print(f"Best num_transformer_blocks: {best_hps.get('num_transformer_blocks')}")
# print(f"Best num_heads: {best_hps.get('num_heads')}")
# print(f"Best ff_dim: {best_hps.get('ff_dim')}")
# print(f"Best number of fully connected layers: {best_hps.get('num_layers_fully')}")
# print(f"Best activation Fully1: {best_hps.get('activation_fully')}")
# print(f"Best layer_dense units: {best_hps.get('layer_dense')}")
# print(f"Best learning rate: {best_hps.get('learning_rate')}")

for i in range(best_hps.get('num_layers_fully') - 1):
    print(f"Best units_dense{i}: {best_hps.get(f'units_dense{i}')}")
    print(f"Best activation_dense{i}: {best_hps.get(f'activation_dense{i}')}")

# Train and save the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, [Y_train[:, 0], Y_train[:, 1], Y_train[:, 2]],
                         epochs=20,
                         validation_data=(X_test_scaled,[Y_test[:, 0], Y_test[:, 1], Y_test[:, 2]]),
                         callbacks=[early_stopping, early_stop_on_high_val_loss])


# Speichern des Modells
model_pfad = os.path.join(Pfad, 'best_model_Transformer_30Trials_Bayesian.h5')
model_pfad_tf = os.path.join(Pfad, 'best_model_Transformer_30Trials_Bayesian.keras')
best_model.save(model_pfad)
best_model.save(model_pfad_tf)


Trial 30 Complete [00h 33m 58s]
val_loss: 2.0471489429473877

Best val_loss So Far: 1.5795693397521973
Total elapsed time: 12h 31m 12s
All available hyperparameters:
{'num_transformer_blocks': 2, 'headsize_0': 64, 'num_heads_0': 6, 'ff_dim_0': 32, 'dropout_rate_1_Nummer0': 0.30000000000000004, 'dropout_rate_2_Nummer0': 0.30000000000000004, 'dropout_rate_3_Nummer0': 0.1, 'activation_cnn_0': 'tanh', 'kernel_size_CNNLayer0': 4, 'num_layers_fully': 1, 'layer_dense': 320, 'activation_fully': 'relu', 'l2_dense': 0.008, 'learning_rate': 0.0011186471256805382, 'headsize_1': 192, 'num_heads_1': 5, 'ff_dim_1': 352, 'dropout_rate_1_Nummer1': 0.30000000000000004, 'dropout_rate_2_Nummer1': 0.2, 'dropout_rate_3_Nummer1': 0.4, 'activation_cnn_1': 'tanh', 'kernel_size_CNNLayer1': 4, 'headsize_2': 288, 'num_heads_2': 4, 'ff_dim_2': 192, 'dropout_rate_1_Nummer2': 0.0, 'dropout_rate_2_Nummer2': 0.4, 'dropout_rate_3_Nummer2': 0.30000000000000004, 'activation_cnn_2': 'relu', 'kernel_size_CNNLayer2': 5, 'he

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 10, 11)    │         22 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     18,059 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 11)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 10, 32)    │      1,440 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 32)    │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 10, 11)    │        363 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10, 11)    │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     45,131 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 10, 11)    │          0 │ dropout_4[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 10, 352)   │     15,840 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 10, 352)   │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 10, 11)    │      3,883 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 10, 11)    │          0 │ conv1d_3[0][0],   │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 89,287 (348.78 KB)

 Trainable params: 89,287 (348.78 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 431s 66ms/step - Verstellweg_Phi_mae: 1.0238 - Verstellweg_X_mae: 0.2451 - Verstellweg_Y_mae: 0.5463 - loss: 1.8936 - val_Verstellweg_Phi_mae: 0.5116 - val_Verstellweg_X_mae: 0.2899 - val_Verstellweg_Y_mae: 0.4545 - val_loss: 1.2769
Epoch 2/20
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 436s 68ms/step - Verstellweg_Phi_mae: 0.3644 - Verstellweg_X_mae: 0.2313 - Verstellweg_Y_mae: 0.2613 - loss: 0.8762 - val_Verstellweg_Phi_mae: 0.4261 - val_Verstellweg_X_mae: 0.2882 - val_Verstellweg_Y_mae: 0.4055 - val_loss: 1.1363
Epoch 3/20
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 436s 68ms/step - Verstellweg_Phi_mae: 0.3049 - Verstellweg_X_mae: 0.2316 - Verstellweg_Y_mae: 0.2237 - loss: 0.7765 - val_Verstellweg_Phi_mae: 0.4311 - val_Verstellweg_X_mae: 0.2897 - val_Verstellweg_Y_mae: 0.3941 - val_loss: 1.1305
Epoch 4/20
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 464s 73ms/step - Verstellweg_Phi_mae: 0.2685 - Verstellweg_X_mae: 0.2305 - Verstellweg_Y_mae: 0.2052 - loss: 0.7199 - val_Verstellweg

ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=h5

In [9]:
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import load_model
# Lade das gespeicherte Modell
loaded_model = load_model('best_model_Transformer_003.h5')

# Überprüfe, ob das Modell korrekt geladen wurde
loaded_model.summary()

# Train and save the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, [Y_train[:, 0], Y_train[:, 1], Y_train[:, 2]],
                         epochs=20,
                         validation_data=(X_val_scaled,[Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                         callbacks=[early_stopping])

# Vorhersagen für den Testdatensatz erstellen
predictions = best_model.predict(X_test_scaled)
X_p, Y_p, Phi_p = predictions

# # Rückskalierung der Vorhersagen
# X_pred = scaler_Y_mm.inverse_transform(X_p)
# Y_pred = scaler_Y_mm.inverse_transform(Y_p)
# Phi_pred = scaler_Y_phi.inverse_transform(Phi_p.reshape(-1, 1)).flatten()

# Vorhersagen den einzelnen Outputs zuordnen
Y_pred_KNR_test = X_p, Y_p, Phi_p

# MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error for Verstellweg_X: {mae_X}")
print(f"Mean Absolute Error for Verstellweg_Y: {mae_Y}")
print(f"Mean Absolute Error for Verstellweg_Phi: {mae_Phi}")

# Validierungsverlust für jeden Output plotten
plt.figure(figsize=(12, 8))
plt.plot(history.history['val_Verstellweg_X_mae'], label='Validation Loss - Verstellweg_X')
plt.plot(history.history['val_Verstellweg_Y_mae'], label='Validation Loss - Verstellweg_Y')
plt.plot(history.history['val_Verstellweg_Phi_mae'], label='Validation Loss - Verstellweg_Phi')
plt.title('Validation Loss for Each Output')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.show()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_model_Transformer_003.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [6]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
import shutil
import os
import json
import gc

def bestes_model(X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels):
      
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    Y_val =np.squeeze(Y_val)
    Y_train_scaled = np.squeeze(Y_train_scaled)
    Y_val_scaled = np.squeeze(Y_val_scaled)
    
    # TensorFlow-Sitzung zurücksetzen
    tf.keras.backend.clear_session()
    gc.collect()
     
    # Transformer Encoder Block
    def transformer_encoder(inputs, head_size, num_heads, ff_dim_1,  dropout_1, dropout_2, dropout_3, kernel_size1, activation_f):
        # Normalization and Attention
        x = layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout_1)(x,x)
        x = layers.Dropout(dropout_2)(x)
        res = x + inputs

        # Feed Forward Part
        x = layers.LayerNormalization(epsilon=1e-6)(res)
        x = layers.Conv1D(filters=ff_dim_1, kernel_size=kernel_size1, activation=activation_f,padding='same')(x)
        x = layers.Dropout(dropout_3)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, padding='same')(x)
        
        return x + res


    input_layer = layers.Input(shape=(10, 11))
        
    x = input_layer
    
    x = transformer_encoder(x, 250, 4, 288, 0, 0, 0.3, 3, 'relu')

    # # Transformer Best Trial 2
    # x = transformer_encoder(x, 64, 6, 32, 0.3, 0.3, 0.1, 4, 'tanh')
    # x = transformer_encoder(x, 192, 5, 352, 0.3, 0.2, 0.4, 4, 'tanh')

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)

    # Fully Connected Layers

    # Tranformer Best Trial 1
    dense_layer = layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.002))(x)
    dense_layer_1 = layers.Dense(448, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(dense_layer)
    dense_layer_2 = layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0))(dense_layer_1)
    dense_layer_3 = layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0))(dense_layer_2)
    
    # Trial 2
    # dense_layer = layers.Dense(320, activation='relu', kernel_regularizer=keras.regularizers.l2(0.008))(x)

    # Output Layers
    X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer_3)
    Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer_3)
    Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer_3)

    outputs = [X_output, Y_output, Phi_output]

    model = keras.Model(inputs=input_layer, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(0.0003269360363083118), 
                    loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                    metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})
    
    # # 2 Trail Hyperparametersuche
    # model = keras.Model(inputs=input_layer, outputs=outputs)
    # model.compile(optimizer=keras.optimizers.Adam(0.0011186471256805382), 
    #                 loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
    #                 metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})
    
    

    model.summary()
    
    # Define early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    Transformer_model = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                            epochs=20,
                            validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                            callbacks=[early_stopping])
    
    # Vorhersagen für den Testdatensatz erstellen
    predictions = model.predict(X_test_scaled)
    X_p, Y_p, Phi_p = predictions
    
      # Error für jedes Label berechnen
    Fehler_X = Y_test[:,0]-X_p[:,0]
    Fehler_Y = Y_test[:,1]-Y_p[:,0]
    Fehler_Phi = Y_test[:,2]-Phi_p[:,0]
    
    # print(Fehler_X)
    # print(Fehler_X.shape)
    
    # Fehler in einen DataFrame konvertieren
    df_Fehler = pd.DataFrame({
        'Label_X': Y_test[:,0],
        'Label_Y': Y_test[:,1],
        'Label_Phi': Y_test[:,2],
        'Fehler_X': Fehler_X,
        'Fehler_Y': Fehler_Y,
        'Fehler_Phi': Fehler_Phi})
    
    mae_X = mean_absolute_error(Y_test[:, 0], X_p)
    mae_Y = mean_absolute_error(Y_test[:, 1], Y_p)
    mae_Phi = mean_absolute_error(Y_test[:, 2], Phi_p)
    
    print(f"Mean Absolute Error for Verstellweg_X: {mae_X}")
    print(f"Mean Absolute Error for Verstellweg_Y: {mae_Y}")
    print(f"Mean Absolute Error for Verstellweg_Phi: {mae_Phi}")
    
    return mae_X, mae_Y, mae_Phi, df_Fehler
    

In [7]:
import random 
from Splitting_Scaling_Function_SkalierungY import Split_Scaling
from Load_Data_for_Modelling_Function import Data_for_Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from Fensterung_Scaling_CNN_ValData_Interpolation_Test import Fensterung_Scale
from sklearn.metrics import mean_absolute_error
import pandas as pd


random.seed(2)
# Generieren einer Liste von 10 eindeutigen zufälligen Ganzzahlen zwischen 0 und 100
Random_numbers = random.sample(range(101), 10)
print(Random_numbers)


MAE_StandardSplit_df = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])
MAE_BlechSplit_df = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])

data = Data_for_Model(0,1800)

# Initialisierung der Variablen für Fehlerdaten
Df_Fehler_append_Standardsplit = pd.DataFrame()
Df_Fehler_append_Blechsplit = pd.DataFrame()


for n in Random_numbers:
    
    X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split=1, window_size=10)
    mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
    MAE_StandardSplit_df = pd.concat([MAE_StandardSplit_df, pd.DataFrame([{'CV':n, 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])], ignore_index=True)
    df_Fehler.insert(loc=0, column='SplitMethode', value='Standardsplit')
    df_Fehler.insert(loc=1, column='CV', value=n)
    Df_Fehler_append_Standardsplit = pd.concat([Df_Fehler_append_Standardsplit,df_Fehler], ignore_index=True)
    

# for n in Random_numbers:
    
#     X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split=2, window_size=10)
#     mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
#     MAE_BlechSplit_df = pd.concat([MAE_BlechSplit_df, pd.DataFrame([{'CV':n, 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])], ignore_index=True)
#     df_Fehler.insert(loc=0, column='SplitMethode', value='Blechsplit')
#     df_Fehler.insert(loc=1, column='CV', value=n)
#     Df_Fehler_append_Blechsplit = pd.concat([Df_Fehler_append_Blechsplit, df_Fehler],ignore_index=True)
    
    
# Ausgabe der DataFrames
print("MAE Standard Split:")
print(MAE_StandardSplit_df)
print("\nMAE Blech Split:")
print(MAE_BlechSplit_df)

[7, 11, 10, 46, 21, 94, 85, 39, 32, 77]
Anzahl der Zeilen im finalen DataFrame: 255600
None
142
254180
(25418, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 10, 11) 
Shape für die Labels der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 1, 3)
Shape für die Labels der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 1, 3)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 10, 11)    │         22 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     47,011 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 11)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 10, 288)   │      9,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 288)   │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 10, 11)    │      3,179 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10, 11)    │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 10)        │          0 │ add_1[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │      5,632 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 448)       │    229,824 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │     14,368 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      1,056 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 311,005 (1.19 MB)

 Trainable params: 311,005 (1.19 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 10, 11)    │         22 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     47,011 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 11)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 10, 288)   │      9,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 288)   │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 10, 11)    │      3,179 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10, 11)    │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 10)        │          0 │ add_1[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │      5,632 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 448)       │    229,824 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │     14,368 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      1,056 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_X       │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Y       │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Verstellweg_Phi     │ (None, 1)         │         33 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 311,005 (1.19 MB)

 Trainable params: 311,005 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 97s 15ms/step - Verstellweg_Phi_mae: 1.0782 - Verstellweg_X_mae: 0.2205 - Verstellweg_Y_mae: 0.4541 - loss: 1.9420 - val_Verstellweg_Phi_mae: 0.3403 - val_Verstellweg_X_mae: 0.1051 - val_Verstellweg_Y_mae: 0.1881 - val_loss: 0.6985
Epoch 2/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 92s 15ms/step - Verstellweg_Phi_mae: 0.3659 - Verstellweg_X_mae: 0.1066 - Verstellweg_Y_mae: 0.2052 - loss: 0.7365 - val_Verstellweg_Phi_mae: 0.2351 - val_Verstellweg_X_mae: 0.1046 - val_Verstellweg_Y_mae: 0.1668 - val_loss: 0.5548
Epoch 3/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - Verstellweg_Phi_mae: 0.2689 - Verstellweg_X_mae: 0.1013 - Verstellweg_Y_mae: 0.1809 - loss: 0.5985 - val_Verstellweg_Phi_mae: 0.2108 - val_Verstellweg_X_mae: 0.1072 - val_Verstellweg_Y_mae: 0.1418 - val_loss: 0.5048
Epoch 4/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 93s 15ms/step - Verstellweg_Phi_mae: 0.2008 - Verstellweg_X_mae: 0.0985 - Verstellweg_Y_mae: 0.1637 - loss: 0.5068 - val_Verstellweg_Phi

In [8]:
Ordner = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\13_ExcelvonDaten_Code\DeepLearning\Transformer\RandomSeed2_Trial1'

Mean_Standard = MAE_StandardSplit_df[['X','y','phi']].mean() 
Mean_Blech = MAE_BlechSplit_df[['X','y','phi']].mean() 
Std_Standard = MAE_StandardSplit_df[['X','y','phi']].std()
Std_Blech = MAE_BlechSplit_df[['X','y','phi']].std()

MAE_StandardSplit = pd.concat([MAE_StandardSplit_df, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Mean_Standard[0], 'y': Mean_Standard[1], 'phi': Mean_Standard[2]}])], ignore_index=True)
MAE_BlechSplit = pd.concat([MAE_BlechSplit_df, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Mean_Blech[0], 'y': Mean_Blech[1], 'phi': Mean_Blech[2]}])], ignore_index=True)
MAE_StandardSplit_comp = pd.concat([MAE_StandardSplit, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Std_Standard[0], 'y': Std_Standard[1], 'phi': Std_Standard[2]}])], ignore_index=True)
MAE_BlechSplit_comp = pd.concat([MAE_BlechSplit, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Std_Blech[0], 'y': Std_Blech[1], 'phi': Std_Blech[2]}])], ignore_index=True)

print(MAE_BlechSplit)
print(MAE_StandardSplit)

Errors_for_CSV = pd.concat([MAE_StandardSplit_comp, MAE_BlechSplit_comp], axis=1)
print(Errors_for_CSV['X'])

Errors_for_CSV.columns = ['CV', 'Datentyp', 'Error', 'X', 'y', 'phi', 'CV1',
       'Datentyp1', 'Error1', 'X1', 'y1', 'phi1']

for Column in Errors_for_CSV.columns:
        Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')
        
for Column in Df_Fehler_append_Standardsplit:
        Df_Fehler_append_Standardsplit[Column] = Df_Fehler_append_Standardsplit[Column].astype(str).str.replace('.', ',')
        
# for Column in Df_Fehler_append_Blechsplit:
#         Df_Fehler_append_Blechsplit[Column] = Df_Fehler_append_Blechsplit[Column].astype(str).str.replace('.', ',')

Df_Fehler_append_Standardsplit.to_csv(f'{Ordner}\Fehler_TransformerNetwork_Standardsplit_RandomSeed2_Trial1_Modell1.csv', index=True, sep=';')

Errors_for_CSV.to_csv(f'{Ordner}\Ergebnisse_TransformerNetwork_Standardsplit_RandomSeed2_Trial1_Modell1.csv', index=True, sep=';')

           CV    Datentyp Error    X    y  phi
0  Mittelwert  Blechsplit   MAE  NaN  NaN  NaN
            CV       Datentyp Error         X         y       phi
0            7  Standardsplit   MAE  0.091292  0.038072  0.075066
1           11  Standardsplit   MAE  0.009893  0.043664  0.034446
2           10  Standardsplit   MAE   0.26024  0.076545  0.066074
3           46  Standardsplit   MAE  0.005212  0.037741  0.029427
4           21  Standardsplit   MAE   0.09566  0.046458  0.045724
5           94  Standardsplit   MAE  0.093469  0.159511  0.045328
6           85  Standardsplit   MAE  0.007638  0.045096  0.041074
7           39  Standardsplit   MAE   0.00889  0.058305  0.057636
8           32  Standardsplit   MAE   0.16904  0.052801  0.037341
9           77  Standardsplit   MAE  0.081039  0.048808  0.048197
10  Mittelwert  Standardsplit   MAE  0.082237    0.0607  0.048031
           X    X
0   0.091292  NaN
1   0.009893  NaN
2    0.26024  NaN
3   0.005212  NaN
4    0.09566  NaN
5   0.

C:\Users\corvi\AppData\Local\Temp\ipykernel_34964\559207859.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')
C:\Users\corvi\AppData\Local\Temp\ipykernel_34964\559207859.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Df_Fehler_append_Standardsplit[Column] = Df_Fehler_append_Standardsplit[Column].astype(str).str.replace('.', ',')


In [6]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
import shutil
import os
import json
import gc
from sklearn.metrics import mean_absolute_error


# TensorFlow-Sitzung zurücksetzen
tf.keras.backend.clear_session()
gc.collect()
    
# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim_1,  dropout_1, dropout_2, dropout_3, kernel_size1, activation_f  ):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_1)(x,x)
    x = layers.Dropout(dropout_2)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim_1, kernel_size=kernel_size1, activation=activation_f,padding='same')(x)
    x = layers.Dropout(dropout_3)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, padding='same')(x)
    
    return x + res


input_layer = layers.Input(shape=(10, 11))
    
x = input_layer

x = transformer_encoder(x,416,5,288,0.2,0.2,0.2, 5,'relu')
x = transformer_encoder(x,480,4,64,0.1,0.2,0.1,2,'relu')
x = transformer_encoder(x,512,6,416,0.2,0.0,0.0,3,'tanh')
x = transformer_encoder(x,256,3,512,0.3,0.4,0.2,3,'relu')
x = transformer_encoder(x,512,3,32,0.4,0.3,0.0,2,'relu')

x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)

# Fully Connected Layers

    # Fully Connected Layers
dense_layer = layers.Dense(96, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.004))(x)
dense_layer_1 = layers.Dense(288, activation='relu', kernel_regularizer=keras.regularizers.l2(0.005))(dense_layer)
dense_layer_2 = layers.Dense(416, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0))(dense_layer_1)
# dense_layer_2 = layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0))(dense_layer_1)
# dense_layer_3 = layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0))(dense_layer_2)

# Output Layers
X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)

outputs = [X_output, Y_output, Phi_output]

model = keras.Model(inputs=input_layer, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(0.00010361629930697395), 
                loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})

model.summary()

# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Transformer_model = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                        epochs=50,
                        validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                        callbacks=[early_stopping])

# Vorhersagen für den Testdatensatz erstellen
predictions1 = model.predict(X_test_scaled)
X_p, Y_p, Phi_p = predictions1

# predictions_int = model.predict(X_test_scaled_int)
# X_int, Y_int, Phi_int = predictions_int

 # MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X}")
print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y}")
print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi}")

 # MAE für den Testdatensatz berechnen
# mae_X_int = mean_absolute_error(Y_test_interpolation[:,0], X_int)
# mae_Y_int = mean_absolute_error(Y_test_interpolation[:,1], Y_int)
# mae_Phi_int = mean_absolute_error(Y_test_interpolation[:,2], Phi_int)

# print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X_int}")
# print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y_int}")
# print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi_int}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 11)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 10, 11)    │         22 │ input_layer[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     97,771 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 11)    │          0 │ dropout_1[0][0],  │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 10, 288)   │     16,128 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 10, 288)   │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 10, 11)    │      3,179 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10, 11)    │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 10, 11)    │     90,251 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 10, 11)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 10, 11)    │          0 │ dropout_4[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 10, 11)    │         22 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 10, 64)    │      1,472 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 10, 64)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 10, 11)    │        715 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 10, 11)    │          0 │ conv1d_3[0][0],   │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 506,669 (1.93 MB)

 Trainable params: 506,669 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 589s 89ms/step - Verstellweg_Phi_mae: 1.3251 - Verstellweg_X_mae: 0.2546 - Verstellweg_Y_mae: 0.7658 - loss: 2.4185 - val_Verstellweg_Phi_mae: 1.3625 - val_Verstellweg_X_mae: 0.4323 - val_Verstellweg_Y_mae: 0.4505 - val_loss: 2.3122
Epoch 2/50
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 572s 90ms/step - Verstellweg_Phi_mae: 0.2978 - Verstellweg_X_mae: 0.2303 - Verstellweg_Y_mae: 0.3406 - loss: 0.9333 - val_Verstellweg_Phi_mae: 1.2201 - val_Verstellweg_X_mae: 0.4323 - val_Verstellweg_Y_mae: 0.6632 - val_loss: 2.3740
Epoch 3/50
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 578s 91ms/step - Verstellweg_Phi_mae: 0.2244 - Verstellweg_X_mae: 0.2334 - Verstellweg_Y_mae: 0.2799 - loss: 0.7946 - val_Verstellweg_Phi_mae: 1.2081 - val_Verstellweg_X_mae: 0.4319 - val_Verstellweg_Y_mae: 0.6638 - val_loss: 2.3562
Epoch 4/50
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 586s 92ms/step - Verstellweg_Phi_mae: 0.1802 - Verstellweg_X_mae: 0.2330 - Verstellweg_Y_mae: 0.2320 - loss: 0.6964 - val_Verstellweg

In [7]:
from sklearn.metrics import mean_absolute_error
 # MAE für den Testdatensatz berechnen
mae_X = mean_absolute_error(Y_test[:,0], X_p)
mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X}")
print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y}")
print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi}")

 # MAE für den Testdatensatz berechnen
mae_X_int = mean_absolute_error(Y_test_interpolation[:,0], X_int)
mae_Y_int = mean_absolute_error(Y_test_interpolation[:,1], Y_int)
mae_Phi_int = mean_absolute_error(Y_test_interpolation[:,2], Phi_int)

print(f"Mean Absolute Error for Verstellweg_X_ alleDaten: {mae_X_int}")
print(f"Mean Absolute Error for Verstellweg_Y_alleDaten: {mae_Y_int}")
print(f"Mean Absolute Error for Verstellweg_Phi_alleDaten: {mae_Phi_int}")

Mean Absolute Error for Verstellweg_X_ alleDaten: 0.22618922188628704
Mean Absolute Error for Verstellweg_Y_alleDaten: 0.37430240199601517
Mean Absolute Error for Verstellweg_Phi_alleDaten: 0.47989635424360483
Mean Absolute Error for Verstellweg_X_ alleDaten: 0.4019775750596957
Mean Absolute Error for Verstellweg_Y_alleDaten: 0.3472218823923037
Mean Absolute Error for Verstellweg_Phi_alleDaten: 0.4339562900588471
